<a href="https://colab.research.google.com/github/gaozhengliu/Causality/blob/master/Data_Generation_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **The purpose of this notebook is to create database tables that meet certain constraints, in order to foster detection of bias**


First dataset: Berkeley admissions data from 1973-1974

We specify the distribution by specific subgroups.


In [ ]:
#given distribution by sub-group

dist = {}
dist["men-deptA-admit"] = (825, 62)
dist["men-deptB-admit"] = (560, 63)
dist["men-deptC-admit"] = (325, 37)
dist["men-deptD-admit"] = (417, 33)
dist["men-deptE-admit"] = (191, 28)
dist["men-deptF-admit"] = (373, 6)

dist["women-deptA-admit"] = (108, 82)
dist["women-deptB-admit"] = (25, 68)
dist["women-deptC-admit"] = (593, 34)
dist["women-deptD-admit"] = (375, 35)
dist["women-deptE-admit"] = (393, 24)
dist["women-deptF-admit"] = (341, 7)

We can compute summary statistics for the overall data distribution.

In [ ]:
totalMen = 0
totalWomen = 0
totalMenAdmitted = 0
totalWomenAdmitted = 0

for sub_group in dist:
  curGroup = sub_group.split("-")
  if curGroup[0] ==  "men":
    totalMen += dist[sub_group][0]
    #multiply admissions number by 0.01 to get percent, and then by # applicants to get total number admitted
    totalMenAdmitted += dist[sub_group][1] * 0.01 * dist[sub_group][0]

  elif curGroup[0] == "women":
    totalWomen  += dist[sub_group][0]
    totalWomenAdmitted += dist[sub_group][1] * 0.01 * dist[sub_group][0]


print("{} men applied and {} men were admitted. {} women applied and {} women were admitted".format(totalMen, round(totalMenAdmitted), totalWomen, round(totalWomenAdmitted)))
print("Overall male admission rate = {maleAdmitP}. Overall female admission rate = {femaleAdmitP}".format(maleAdmitP = round(totalMenAdmitted/totalMen, 2), femaleAdmitP = round(totalWomenAdmitted/totalWomen, 2)))

2691 men applied and 1198 men were admitted. 1835 women applied and 557 women were admitted
Overall male admission rate = 0.45. Overall female admission rate = 0.3


Next, we create a list in python which satisfies the various sub-group distributions, and shuffle it to make it look more authentic.

In [ ]:
reformatter = {"men": "man", "women": "woman"} #intended to make certain text nicer


records = []
for sub_group in dist:
   gender, dept, admit_rate = sub_group.split("-")
   dept = dept.replace("dept", "")
   gender = reformatter[gender]
   genderCode = ""
   if gender == "woman" or gender == "women":
     genderCode = "F"
   elif gender == "man" or gender == "men":
     genderCode = "M"

   #we only stored admit percent. to get admit number, need to multiply admit percent by 0.01 and by number who applied
   num_applicants, num_admits = dist[sub_group][0], round(dist[sub_group][1] * 0.01 * dist[sub_group][0])

   not_admitted = num_applicants - num_admits
   
   for i in range(num_applicants):
     unique_name = genderCode + str(i) + dept.replace("dept","")

     if i < num_admits:
        records.append([unique_name, genderCode, dept, "Y"]) #Y stands for admitted.
     else:
       records.append([unique_name, genderCode, dept, "N"])


import random

random.seed(10)
random.shuffle(records)

In [ ]:
#print out records or save to csv
for i in range(10):
  print(records[i])

['M46A', 'M', 'A', 'Y']
['M364B', 'M', 'B', 'N']
['M158A', 'M', 'A', 'Y']
['M453B', 'M', 'B', 'N']
['F248C', 'F', 'C', 'N']
['M399A', 'M', 'A', 'Y']
['M771A', 'M', 'A', 'N']
['F97E', 'F', 'E', 'N']
['M53D', 'M', 'D', 'Y']
['M705A', 'M', 'A', 'N']


Now, we store the data in a sqlite database for easy access

In [ ]:

import sqlite3


make_table_statement = """CREATE TABLE IF NOT EXISTS ADMISSIONS (
  id integer PRIMARY KEY AUTOINCREMENT, 
  name text NOT NULL, 
  gender text NOT NULL, 
  department text NOT NULL, 
  decision text NOT NULL
);"""

try:
    sqliteConnection = sqlite3.connect('SQLite_Python.db')
    cursor = sqliteConnection.cursor()

    cursor.execute("DROP TABLE IF EXISTS ADMISSIONS") #clear the table and make fresh
    cursor.execute(make_table_statement)
    
    print("Database created and Successfully Connected to SQLite")
    sqliteConnection.commit()


    for record in records:
      name, gender, department, decision = record[0], record[1], record[2], record[3]
    
    
      
      cursor.execute("INSERT INTO ADMISSIONS(name, gender, department, decision) VALUES(?,?,?,?)", (name, gender, department, decision))
    
    sqliteConnection.commit()
    cursor.close()
    print("Done saving")
except:
    print("DB operation failed")
    pass







Database created and Successfully Connected to SQLite
Done saving


The database with the admissions table should have been successfully created at this point. You can download it and play with it using a DBMS. Here is one online: https://sqliteonline.com/